In [ ]:
#packages
import numpy as np
import scipy 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pygam
import sklearn
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

print(f'Package versions: \n')
print(f'numpy {np.__version__}')
print(f'scipy {scipy.__version__}')
print(f'pandas {pd.__version__}')
print(f'seaborn {sns.__version__}')
print(f'pygam {pygam.__version__}')
print(f'scikit-learn {sklearn.__version__}')
print(f'optuna {optuna.__version__}')

import warnings
warnings.filterwarnings('ignore') 

In [ ]:
df_olist_customers_dataset = pd.read_csv('/Users/guojingwei/Documents/USYD/6810/group assignment/Olist Dataset/olist_customers_dataset.csv')
df_olist_geolocation_dataset = pd.read_csv('/Users/guojingwei/Documents/USYD/6810/group assignment/Olist Dataset/olist_geolocation_dataset.csv')
df_olist_order_items_dataset = pd.read_csv('/Users/guojingwei/Documents/USYD/6810/group assignment/Olist Dataset/olist_order_items_dataset.csv')
df_olist_order_payments_dataset = pd.read_csv('/Users/guojingwei/Documents/USYD/6810/group assignment/Olist Dataset/olist_order_payments_dataset.csv')
df_olist_order_reviews_dataset = pd.read_csv('/Users/guojingwei/Documents/USYD/6810/group assignment/Olist Dataset/olist_order_reviews_dataset.csv')
df_olist_orders_dataset = pd.read_csv('/Users/guojingwei/Documents/USYD/6810/group assignment/Olist Dataset/olist_orders_dataset.csv')
df_olist_products_dataset = pd.read_csv('/Users/guojingwei/Documents/USYD/6810/group assignment/Olist Dataset/olist_products_dataset.csv')
df_olist_sellers_dataset = pd.read_csv('/Users/guojingwei/Documents/USYD/6810/group assignment/Olist Dataset/olist_sellers_dataset.csv')
df_product_category_name_translation = pd.read_csv('/Users/guojingwei/Documents/USYD/6810/group assignment/Olist Dataset/product_category_name_translation.csv')

df_olist_order_reviews_dataset = df_olist_order_reviews_dataset.rename(columns={'review_id': 'customer_id'})

df_temp = pd.merge(df_olist_order_items_dataset, df_olist_order_payments_dataset, on='order_id')
df_order_based = pd.merge(df_temp, df_olist_orders_dataset, on='order_id')

df_order_medium = pd.merge(df_order_based, df_olist_products_dataset, on='product_id')
df_order_final = pd.merge(df_order_medium, df_olist_sellers_dataset, on='seller_id')
df_customer = pd.merge(df_olist_customers_dataset, df_olist_orders_dataset, on='customer_id')
df_merged = pd.merge(df_order_final, df_customer, on='order_id')

df_merged.info()
print(df_merged.head(20))

In [ ]:
df_AOV = df_merged.groupby('customer_unique_id').agg(
    total_order_amount=('price', 'sum'),
    order_count=('order_id', 'count')
).reset_index()

df_AOV['AOV'] = df_AOV['total_order_amount'] / df_AOV['order_count']




df_merged_with_AOV = pd.merge(df_merged, df_AOV, on='customer_unique_id')
print(df_merged_with_AOV)

response = ['AOV']
continuous = [
    'price', 'freight_value', 'payment_value',
    'product_name_lenght', 'product_description_lenght',
    'product_photos_qty', 'product_weight_g',
    'product_length_cm', 'product_height_cm', 'product_width_cm'
]

discrete = [
    'payment_installments', 'payment_sequential'
]

binary = [
    'payment_type', 'product_category_name', 'seller_state', 'customer_state'
]

predictors = continuous + discrete + binary
    
df_merged_with_AOV = df_merged_with_AOV[response + predictors]


X = df_merged_with_AOV[predictors]
y = df_merged_with_AOV['AOV']

In [ ]:
print(train[predictors].info())
print(train[predictors].head())

In [ ]:
print(len(train[predictors]))
print(len(y_train))
print(pd.Series(y_train).isna().sum())

In [ ]:
print(train[predictors].describe())

In [ ]:
# Randomly split indexes
index_train, index_valid  = train_test_split(np.array(df_merged_with_AOV.index), train_size=0.7, random_state=1)

# Write training and validation sets 
train = df_merged_with_AOV.loc[index_train,:].copy()
valid =  df_merged_with_AOV.loc[index_valid,:].copy()

y_train= train['AOV'].to_numpy()
y_valid = valid['AOV'].to_numpy()


from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train.loc[:, continuous + discrete] = scaler.fit_transform(train.loc[:, continuous + discrete])
valid.loc[:, continuous + discrete] = scaler.transform(valid.loc[:, continuous + discrete])

X_train = train[predictors]
X_valid = valid[predictors]


In [ ]:
def plot_relationships(X, y, lowess=False):
    """Plot the relationship between each predictor and the target variable."""
    # Feature names and setup
    features = X.columns
    num_features = len(features)
    rows = (num_features + 2) // 3  # Calculate rows needed (3 plots per row)
    
    # Create figure
    fig, axes = plt.subplots(rows, 3, figsize=(15, rows * 4))
    axes = axes.flatten()  # Flatten to easily iterate
    
    # Plot each feature
    for i, feature in enumerate(features):
        if i < len(axes):
            # Scatter with smoothed line
            sns.regplot(x=X[feature], y=y, lowess=lowess, 
                       scatter_kws={'alpha': 0.5}, ax=axes[i])
            axes[i].set_title(f'{feature} vs AOV')
            axes[i].set_xlabel(feature)
            axes[i].set_ylabel('AOV')
    
    # Remove any unused subplots
    for i in range(num_features, len(axes)):
        fig.delaxes(axes[i])
    
    plt.tight_layout()
    return fig
fig = plot_relationships(train[predictors], y_train, lowess=True)
plt.show()

In [ ]:
def fit_and_plot_spline(feature, n_knots, degree, y_train, train_data):
    """Fit a spline with specified parameters and plot the results."""
    # Create the spline transformer
    spline = SplineTransformer(n_knots=n_knots, degree=degree, knots='uniform', include_bias=False)
    X_spline = spline.fit_transform(train_data[[feature]].values)
    
    # Fit linear regression on spline features
    model = LinearRegression()
    model.fit(X_spline, y_train)
    
    # Create grid for visualization
    x_min = train_data[feature].min()
    x_max = train_data[feature].max()
    x_grid = np.linspace(x_min, x_max, 100).reshape(-1, 1)
    X_grid = spline.transform(x_grid)
    
    # Make predictions
    y_pred = model.predict(X_grid)
    
    # Plot
    plt.figure(figsize=(10, 6))
    plt.scatter(train_data[feature], y_train, alpha=0.4, label='Data')
    plt.plot(x_grid, y_pred, linewidth=2, 
             label=f'degree {degree} spline with {n_knots} knots')
    plt.xlabel(f'{feature} (standardized)')
    plt.ylabel('Customer Lifetime Value')
    plt.title(f'Regression Spline: Degree {degree} with {n_knots} knots')
    plt.legend()
    plt.show()
    
    return model
# Linear spline with 4 knots
fit_and_plot_spline('total_order_amount', n_knots=4, degree=1, y_train=y_train, train_data=train)

# Cubic spline with 5 knots
fit_and_plot_spline('total_order_amount', n_knots=5, degree=3, y_train=y_train, train_data=train)

In [ ]:
def fit_and_plot_spline(feature, n_knots, degree, y_train, train_data):
    """Fit a spline with specified parameters and plot the results."""
    # Create the spline transformer
    spline = SplineTransformer(n_knots=n_knots, degree=degree, knots='uniform', include_bias=False)
    X_spline = spline.fit_transform(train_data[[feature]].values)
    
    # Fit linear regression on spline features
    model = LinearRegression()
    model.fit(X_spline, y_train)
    
    # Create grid for visualization
    x_min = train_data[feature].min()
    x_max = train_data[feature].max()
    x_grid = np.linspace(x_min, x_max, 100).reshape(-1, 1)
    X_grid = spline.transform(x_grid)
    
    # Make predictions
    y_pred = model.predict(X_grid)
    
    # Plot
    plt.figure(figsize=(10, 6))
    plt.scatter(train_data[feature], y_train, alpha=0.4, label='Data')
    plt.plot(x_grid, y_pred, linewidth=2, 
             label=f'degree {degree} spline with {n_knots} knots')
    plt.xlabel(f'{feature} (standardized)')
    plt.ylabel('Customer Lifetime Value')
    plt.title(f'Regression Spline: Degree {degree} with {n_knots} knots')
    plt.legend()
    plt.show()
    
    return model
from pygam import LinearGAM, s, l

# Define the model: s() for smooth terms, l() for linear terms
gam = LinearGAM(
    s(0) +  # Acq_Expense: smooth term
    s(1) +  # Ret_Expense: smooth term 
    l(2) +  # First_Purchase: linear term
    l(3) +  # Revenue: linear term
    l(4) +  # Employees: linear term
    l(5) +  # Frequency: linear term
    l(6) +  # Crossbuy: linear term
    l(7)    # Industry: linear term
)

# Fit the model
gam.fit(X_train, y_train)

# Display model summary
print(gam.summary())

In [ ]:
# Plotting the partial dependence for each feature
plt.figure(figsize=(16, 20))

# Track the plot position
plot_idx = 1

for i, term in enumerate(gam.terms):
    if term.isintercept:
        continue
        
    # Create subplot (2 plots per row)
    plt.subplot(4, 2, plot_idx)
    plot_idx += 1
    
    # Generate points for plotting
    X_grid = gam.generate_X_grid(term=i)
    
    # Get the partial dependence and confidence intervals
    pdep, confi = gam.partial_dependence(term=i, X= X_grid, width=0.95)
    
    # Plot the function
    plt.plot(X_grid[:, term.feature], pdep)
    plt.plot( X_grid[:, term.feature], confi, c='grey', ls='--', alpha=0.5)
    
    # Label the plot
    plt.title(predictors[i])
    plt.grid(True, alpha=0.2)

plt.tight_layout()
plt.show()

In [ ]:
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING) # Swithing off ouput, you want to remove this line to see more details

# Define a simple objective function to minimize
def objective(trial):
    # Parameter to optimize
    x = trial.suggest_float('x', -10, 10)
    
    # The function to minimize: (x-2)²
    return (x - 2) ** 2

# Create a study object and optimize
sampler = optuna.samplers.TPESampler(seed=42)  # For reproducibility
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=100)

# Print results
print(f"Best value: {study.best_value:.4f}")
print(f"Best parameters: {study.best_params}")

In [ ]:
def gam_objective(trial):
    # Separate lambda parameters for different term types
    lambda_spline_acq = trial.suggest_float('lambda_spline_acq', 1e-4, 1e4, log=True)
    lambda_spline_ret = trial.suggest_float('lambda_spline_ret', 1e-4, 1e4, log=True)
    lambda_linear = trial.suggest_float('lambda_linear', 1e-4, 1e4, log=True)
    
    # Create lambda list with appropriate values for each term
    lambdas = [
        lambda_spline_acq,  # Acq_Expense (spline)
        lambda_spline_ret,  # Ret_Expense (spline)
        lambda_linear,   # First_Purchase (linear)
        lambda_linear,   # Revenue (linear)
        lambda_linear,   # Employees (linear)
        lambda_linear,   # Frequency (linear)
        lambda_linear,   # Crossbuy (linear)
        lambda_linear    # Industry (linear)
    ]
    
    # Create and fit the GAM with these lambda values
    model = LinearGAM(s(0) + s(1) + l(2) + l(3) + l(4) + l(5) + l(6) + l(7), 
                      lam=lambdas)
    model.fit(X_train, y_train)
    
    # Return the GCV score
    return model.statistics_['GCV']

sampler = optuna.samplers.TPESampler(seed=42)  # makes the sampler behave in a deterministic way

study = optuna.create_study(direction='minimize', sampler=sampler) # TPE is a method for Bayesian optimisation
study.optimize(gam_objective, n_trials=10000, timeout=120) 

In [ ]:
best_lambdas = list(study.best_params.values())

lambda_spline_acq = best_lambdas[0]
lambda_spline_ret = best_lambdas[1]
lambda_linear = best_lambdas[2]
    
# Create lambda list with appropriate values for each term
lambdas = [
    lambda_spline_acq,  # Acq_Expense (spline)
    lambda_spline_ret,  # Ret_Expense (spline)
    lambda_linear,   # First_Purchase (linear)
    lambda_linear,   # Revenue (linear)
    lambda_linear,   # Employees (linear)
    lambda_linear,   # Frequency (linear)
    lambda_linear,   # Crossbuy (linear)
    lambda_linear    # Industry (linear)
]


optimized_gam = LinearGAM(s(0) + s(1) + l(2) + l(3) + l(4) + l(5) + l(6) + l(7), lam = lambdas)
optimized_gam.fit(X_train, y_train)

In [ ]:
print(gam.summary())

In [ ]:
plt.figure(figsize=(16, 20))
plot_idx = 1

for i, term in enumerate(optimized_gam.terms):
    if term.isintercept:
        continue
        
    plt.subplot(4, 2, plot_idx)
    plot_idx += 1
    
    X_grid = optimized_gam.generate_X_grid(term=i)
    pdep, confi = optimized_gam.partial_dependence(term=i, X=X_grid, width=0.95)
    
    plt.plot(X_grid[:, term.feature], pdep)
    plt.plot(X_grid[:, term.feature], confi, c='grey', ls='--', alpha=0.5)
    
    plt.title(f"{predictors[i]}")
    plt.grid(True, alpha=0.2)
    plt.ylim(-1.5, 1.5) 

plt.tight_layout()
plt.show()
